In [ ]:
SYFT_VERSION = ">=0.9,<1.0.0"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [ ]:
# third party
import torch
import torch.nn as nn
import torch.nn.functional as F

# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

In [ ]:
server = sy.orchestra.launch(name="test-datasite-1", dev_mode=True, reset=True)

In [ ]:
datasite_client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
# Set the random seed for reproducibility
torch.manual_seed(42)

In [ ]:
# Generate random data
train_data = torch.rand((4, 28, 28, 1))
train_data.shape

In [ ]:
assert torch.round(train_data.sum()) == 1557

In [ ]:
train = sy.ActionObject.from_obj(train_data)

In [ ]:
type(train.syft_action_data), train.id, train.shape

In [ ]:
train_datasite_obj = train.send(datasite_client)
type(train_datasite_obj)

In [ ]:
train_datasite_obj

In [ ]:
assert torch.round(train_datasite_obj.syft_action_data.sum()) == 1557

In [ ]:
class MLP(nn.Module):
    def __init__(self, out_dims):
        super().__init__()
        self.out_dims = out_dims
        self.linear1 = nn.Linear(784, 128)
        self.linear2 = nn.Linear(128, out_dims)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x


model = MLP(out_dims=10)
model

In [ ]:
weights = model.state_dict()

In [ ]:
assert isinstance(weights, dict)

In [ ]:
w = sy.ActionObject.from_obj(weights)

In [ ]:
type(w.syft_action_data), w.id

In [ ]:
weight_datasite_obj = w.send(datasite_client)

In [ ]:
@sy.syft_function(
    input_policy=sy.ExactMatch(
        weights=weight_datasite_obj.id, data=train_datasite_obj.id
    ),
    output_policy=sy.SingleExecutionExactOutput(),
)
def train_mlp(weights, data):
    # third party
    import torch
    import torch.nn as nn
    import torch.nn.functional as F

    class MLP(nn.Module):
        def __init__(self, out_dims):
            super().__init__()
            self.out_dims = out_dims
            self.linear1 = nn.Linear(784, 128)
            self.linear2 = nn.Linear(128, out_dims)

        def forward(self, x):
            x = x.view(x.size(0), -1)
            x = self.linear1(x)
            x = F.relu(x)
            x = self.linear2(x)
            return x

    # Initialize the model
    model = MLP(out_dims=10)

    # Load weights into the model
    model.load_state_dict(weights)

    # Perform a forward pass
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        output = model(data)

    return output

In [ ]:
pointer = train_mlp(weights=weight_datasite_obj, data=train_datasite_obj)
output = pointer.get()

In [ ]:
assert torch.allclose(torch.sum(output), torch.tensor(1.3907))

In [ ]:
request = datasite_client.code.request_code_execution(train_mlp)
request

In [ ]:
request.approve()

In [ ]:
datasite_client._api = None
_ = datasite_client.api

In [ ]:
result_ptr = datasite_client.code.train_mlp(weights=w.id, data=train.id)

In [ ]:
result = result_ptr.get()

In [ ]:
assert torch.allclose(torch.sum(result), torch.tensor(1.3907))

In [ ]:
if server.server_type.value == "python":
    server.land()